# Image processing for material property estimation

### Compaction experiment with CT images

This notebook contains the exact same steps as in `materials_testing_notebook.py`, with the following differences:
* the content only goes up to the point of processing input images - in this case, **grayscale** images - into a two-phase (binary) material
* here, we import the `Microstructure` class and methods from the `uumatsci_utils` library (included with this file)
* some cells have been completely deleted - they were for stand-alone testing and irrelevant in the context of the example

In [54]:
# import the uumatsci_utils.py library
import uumatsci_utils as uumat
import numpy as np
import os

#### Step 1: loading image files

Here, we load the grayscale image files containing the raw CT images - each image in this case corresponds to a different stress state over the compaction experiment. 

In [55]:
import matplotlib
matplotlib.use('TkAgg')
# matplotlib.use('Qt5Agg')
# matplotlib.use('MACOSX')
import matplotlib.pyplot as plt

# %% set paths
path = 'D:/SerpRateAI/Results'
image_number='CS_5057_5_B_12_1_1'
image_file = path + '/' + image_number +'/' + image_number +'_Simple Segmentation_3.tif'


# %% alternative image read method
img_ct = plt.imread(image_file)
print(img_ct.shape)

plt.figure()
plt.imshow(img_ct, cmap=plt.cm.gray)
plt.show()

(503, 1005)


#### Step 2: convert images into microstructure

In the `uumatsci_utils.py` library, we define a class called `Microstructure` - that object contains:
* the actual microstructure information: both the binary two-phase structure, and the final processed image from which said microstructure is estimated
* methods - i.e., class-based functions - to calculate microstructure descriptors

For now, let's focus on one method of the `Microstructure` class - called `twoDCTimage2structure`: this is designed to take a 2D grayscale image (such as from a CT scanner) into a two-phase microstructure, that later will be used to estimate microstructure descriptors.

In [57]:
### Example image 1

height=img_ct.shape[0]
width=img_ct.shape[1]
print(width, height)
if width >= (2*height//3):
    y_start = 10
else:
    y_start=height//3
x_start=10
x_size=min(width, height)-20
y_size=x_size

# this 'par' is a dictionary with the arguments passed to the image-to-structure method
par={'name':image_number,'begx': y_start, 'begy': x_start, 'nsampx': x_size, 'nsampy': y_size, 'edge_buffer': 10,
    'equalisation': False, 'equal_method': 'adaptive', 'stretch_percentile': 2,
    'clip_limit': 0.03, 'tvdnoise': True, 'tv_weight': 0.15, 'tv_eps': 2e-04,
    'median_filter': False, 'median_filter_length': 3,
    'thresholding_method': 'otsu', 'thresholding_weight': 0.85, 'nbins': 256,
    'make_figs': True, 'fig_res': 400, 'fig_path':'D:/SerpRateAI/'}


# test function
imct_microstructure = uumat.twoDCTimage2structure(img_ct, par)
print(type(imct_microstructure))
imm = imct_microstructure.sourceimage
print(imm.shape)
#img_adaptive = exposure.equalize_adapthist(imm, clip_limit=par['clip_limit'])

# test volume fraction calculation

imct_microstructure.volumefraction()
imct_nincl = imct_microstructure.ninclusion
imct_phi = imct_microstructure.volfracvalue
print("Number of inclusions: %s" % imct_nincl)
print("Volume fraction: %s" % imct_phi)

# test listing inclusion indeces
imct_microstructure.list_inclusion_indeces()
imct_inclist = imct_microstructure.inclusion_index_list
print(imct_inclist.shape)
for i in range(10):
    print("%s  %s" % (imct_inclist[0, i], imct_inclist[1, i]))

#plt.figure()
#plt.imshow(imct_microstructure.structure, cmap=plt.cm.gray)

1005 503
(503, 503)
9 492
<class 'uumatsci_utils.Microstructure'>
(483, 483)
Number of inclusions: 13537
Volume fraction: 0.05802673936619386
(2, 13537)
18  226
18  227
18  228
18  229
18  231
18  232
18  233
18  247
18  248
19  226


#### Step 3: SMD calculations

In the `uumatsci_utils.py` library, there is a method called `estimate_npolytope_functions` - which:
* runs the **C++** executable `Sample_Pn_UU`: this calculates S2 and Polytope functions
* manipulates the necessary I/O files in a way that attempts to protects subsenquent calls of this method not to get mixed up with input/information from previous calls

**Important:** the C++ code must be compiled separately, and the parameters `file_path`, `cpp_path` and `runtime_path` **must be set carefully** to ensure result files are properly documented nad kept separately.

Below are the initial results using the *Slochteren compaction experiment data from Suzanne Hangx*. Plese note these results are unpublished and therefore **confidential**.

In [45]:
# Set up to run C++ polytope codes
# test writing Mconfig files
if os.path.isfile('D:/SerpRateAI/Connectivity/PyMMat-master/runtime/Mconfig.txt'):
    os.remove('D:/SerpRateAI/Connectivity/PyMMat-master/runtime/Mconfig.txt')

if os.path.isfile('D:/SerpRateAI/Connectivity/PyMMat-master/runtime/output/' + image_number +'_Mconfig.txt'):
    os.remove('D:/SerpRateAI/Connectivity/PyMMat-master/runtime/output/' + image_number +'_Mconfig.txt')


cpathPn = 'D:/SerpRateAI/Connectivity/PyMMat-master/Cpp_source/Polytope/'
runtimePn = 'D:/SerpRateAI/Connectivity/PyMMat-master/runtime/'
outputPn = 'D:/SerpRateAI/Connectivity/PyMMat-master/runtime/output/'

# test polytope estimations
imct_microstructure.estimate_npolytope_functions(file_path=outputPn, cppcode_path=cpathPn, runtime_path=runtimePn, verbose=False)
imct_PnS2 = imct_microstructure.polytope_S2
print(imct_PnS2.shape)
print(imct_PnS2[0:10,:])



Writing CS_5057_5_B_124_4_1_Mconfig.txt file in: D:/SerpRateAI/Connectivity/PyMMat-master/runtime/output/
Writing Mconfig.txt file for sample CS_5057_5_B_124_4_1 in current directory
(250, 2)
[[0.000e+00 5.558e-03]
 [1.000e+00 4.197e-03]
 [2.000e+00 3.145e-03]
 [3.000e+00 2.384e-03]
 [4.000e+00 1.841e-03]
 [5.000e+00 1.498e-03]
 [6.000e+00 1.301e-03]
 [7.000e+00 1.171e-03]
 [8.000e+00 1.062e-03]
 [9.000e+00 9.800e-04]]


In [46]:
# plot Pn for the sample
import os

print(os.getcwd())
rows = 1
cols = 1
figsize = (16, 6)
fig, axes = plt.subplots(rows, cols, sharey=True, figsize=figsize, constrained_layout=True)

axes.set_title('Pn: %s' % imct_microstructure.name)
axes.set_ylabel('Pn')
axes.set_xlabel('r (pixels)')
axes.plot(imct_PnS2[:,0], imct_PnS2[:,1], 'o', ls='-', ms=4, markevery=None, label='S2')
axes.plot(imct_microstructure.polytope_L[:,0], imct_microstructure.polytope_L[:,1], 'o', ls='-', ms=4, markevery=None, label='L')
axes.plot(imct_microstructure.polytope_P3V[::2,0], imct_microstructure.polytope_P3V[::2,1], 'o', ls='-', ms=4, markevery=None, label='P3V')
axes.plot(imct_microstructure.polytope_P3H[::2,0], imct_microstructure.polytope_P3H[::2,1], 'o', ls='-', ms=4, markevery=None, label='P3H')
axes.plot(imct_microstructure.polytope_P4[:,0], imct_microstructure.polytope_P4[:,1], 'o', ls='-', ms=4, markevery=None, label='P4')
axes.plot(imct_microstructure.polytope_P6V[::2,0], imct_microstructure.polytope_P6V[::2,1], 'o', ls='-', ms=4, markevery=None, label='P6V')

axes.grid(True)
axes.legend()

plt.savefig('D:/SerpRateAI/Connectivity/Polytope_results/'+ image_number + '_test.tif', dpi=400)



D:\SerpRateAI\Connectivity\PyMMat-master


In [47]:
## Calculate f and fn functions
imct_microstructure.calculate_scaled_autocovariance()
imct_microstructure.calculate_polytope_fn()

In [48]:
# plot Pn
print(os.getcwd())
rows = 1
cols = 1
figsize = (16, 10)
fig, axes = plt.subplots(rows, cols, sharey=True, figsize=figsize, constrained_layout=True)


axes.set_title('Pn: %s' % imct_microstructure.name)
axes.set_ylabel('Pn')
axes.set_xlabel('r (pixels)')
axes.plot(imct_microstructure.polytope_S2[0:50,0], imct_microstructure.polytope_S2[0:50,1], 'o', ls='-', ms=4, markevery=None, label='S2')
axes.plot(imct_microstructure.polytope_L[0:50,0], imct_microstructure.polytope_L[0:50,1], 'o', ls='-', ms=4, markevery=None, label='L')
axes.plot(imct_microstructure.polytope_P3V[0:50:2,0], imct_microstructure.polytope_P3V[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P3V')
axes.plot(imct_microstructure.polytope_P3H[0:50:2,0], imct_microstructure.polytope_P3H[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P3H')
axes.plot(imct_microstructure.polytope_P4[0:50,0], imct_microstructure.polytope_P4[0:50,1], 'o', ls='-', ms=4, markevery=None, label='P4')
axes.plot(imct_microstructure.polytope_P6V[0:50:2,0], imct_microstructure.polytope_P6V[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P6V')

axes.grid(True)
axes.legend()


plt.savefig('D:/SerpRateAI/Connectivity/Polytope_results/Pn_r50_' +image_number +'.tif', dpi=400)

D:\SerpRateAI\Connectivity\PyMMat-master


In [49]:
# plot Fn
print(os.getcwd())
rows = 1
cols = 1
figsize = (16, 10)
fig, axes = plt.subplots(rows, cols, sharey=True, figsize=figsize, constrained_layout=True)


axes.set_title('Fn: %s' % imct_microstructure.name)
axes.set_ylabel('$f_n$')
axes.set_xlabel('r (pixels)')
axes.plot(imct_microstructure.scal_autocov[0:50,0], imct_microstructure.scal_autocov[0:50,1], 'o', ls='-', ms=4, markevery=None, label='S2')
axes.plot(imct_microstructure.polyfn_L[0:50,0], imct_microstructure.polyfn_L[0:50,1], 'o', ls='-', ms=4, markevery=None, label='L')
axes.plot(imct_microstructure.polyfn_P3V[0:50:2,0], imct_microstructure.polyfn_P3V[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P3V')
axes.plot(imct_microstructure.polyfn_P3H[0:50:2,0], imct_microstructure.polyfn_P3H[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P3H')
axes.plot(imct_microstructure.polyfn_P4[0:50,0], imct_microstructure.polyfn_P4[0:50,1], 'o', ls='-', ms=4, markevery=None, label='P4')
axes.plot(imct_microstructure.polyfn_P6V[0:50:2,0], imct_microstructure.polyfn_P6V[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P6V')

axes.grid(True)
axes.legend()


plt.savefig('D:/SerpRateAI/Connectivity/Polytope_results/Fn_r50_' +image_number +'.tif', dpi=400)



D:\SerpRateAI\Connectivity\PyMMat-master


In [50]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
from skimage import exposure

figsize = (16, 8)
fig = plt.figure(figsize=figsize, constrained_layout=True)
gs = fig.add_gridspec(5, 6)

Rects = []
Rect1 = Rectangle((100, 901), 501, 501, fill=False, edgecolor='Green')
Rects.append(Rect1)
pc = PatchCollection(Rects)

f_ax1 = fig.add_subplot(gs[1:5, 1])
f_ax1.set_title('Segmented ' + image_number)
f_ax1.set_ylabel('x (pixels)')
f_ax1.set_xlabel('y (pixels)')
f_ax1.imshow(exposure.equalize_adapthist(img_ct, clip_limit=par['clip_limit']), cmap=plt.cm.gray)
f_ax1.plot([x_start,x_start,x_start+x_size,x_start+x_size,x_start],[y_start,y_start+y_size,y_start+y_size,y_start,y_start],'b',lw=3)


f_ax4 = fig.add_subplot(gs[1:3, 3:5])
f_ax4.set_title('Polytope Pn')
f_ax4.set_ylabel('Pn')
f_ax4.set_xlabel('r (pixels)')
f_ax4.plot(imct_microstructure.polytope_S2[0:50,0], imct_microstructure.polytope_S2[0:50,1], 'o', ls='-', ms=4, markevery=None, label='S2')
f_ax4.plot(imct_microstructure.polytope_P3V[0:50:2,0], imct_microstructure.polytope_P3V[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P3V')
f_ax4.plot(imct_microstructure.polytope_P3H[0:50:2,0], imct_microstructure.polytope_P3H[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P3H')
f_ax4.plot(imct_microstructure.polytope_P4[0:50,0], imct_microstructure.polytope_P4[0:50,1], 'o', ls='-', ms=4, markevery=None, label='P4')
f_ax4.plot(imct_microstructure.polytope_P6V[0:50:2,0], imct_microstructure.polytope_P6V[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P6V')
f_ax4.plot(imct_microstructure.polytope_L[0:50,0], imct_microstructure.polytope_L[0:50,1], 'o', ls='-', ms=4, markevery=None, label='L')
f_ax4.set_ylim([-0.025, 0.3])
f_ax4.grid(True)
f_ax4.legend()

f_ax5 = fig.add_subplot(gs[3:5, 3:5])
f_ax5.set_title('Polytope Fn')
f_ax5.set_ylabel('$f_n$')
f_ax5.set_xlabel('r (pixels)')
f_ax5.plot(imct_microstructure.scal_autocov[0:50,0], imct_microstructure.scal_autocov[0:50,1], 'o', ls='-', ms=4, markevery=None, label='S2')
f_ax5.plot(imct_microstructure.polyfn_P3V[0:50:2,0], imct_microstructure.polyfn_P3V[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P3V')
f_ax5.plot(imct_microstructure.polyfn_P3H[0:50:2,0], imct_microstructure.polyfn_P3H[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P3H')
f_ax5.plot(imct_microstructure.polyfn_P4[0:50,0], imct_microstructure.polyfn_P4[0:50,1], 'o', ls='-', ms=4, markevery=None, label='P4')
f_ax5.plot(imct_microstructure.polyfn_P6V[0:50:2,0], imct_microstructure.polyfn_P6V[0:50:2,1], 'o', ls='-', ms=4, markevery=None, label='P6V')
f_ax5.plot(imct_microstructure.polyfn_L[0:50,0], imct_microstructure.polyfn_L[0:50,1], 'o', ls='-', ms=4, markevery=None, label='L')
f_ax5.set_ylim([-0.025, 1.1])
f_ax5.grid(True)
f_ax5.legend()

plt.savefig('D:/SerpRateAI/Connectivity/Polytope_results/Visualize_results_' + image_number + '_400dpi.tif', dpi=400)
plt.savefig('D:/SerpRateAI/Connectivity/Polytope_results/Visualize_results_' + image_number + '_200dpi.tif', dpi=200)